# 模型推理 - 使用 QLoRA 微调后的 ChatGLM3-6B

In [1]:
import os

os.environ['http_proxy'] = 'http://127.0.0.1:1087'
os.environ['https_proxy'] = 'http://127.0.0.1:1087'
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
peft_model_path_1 = f"/home/cc/projects/llm-quickstart-gcc/week3/models/chatglm3-6b-10k-linear-model"
peft_model_path_2 = f"/home/cc/projects/llm-quickstart-gcc/week3/models/chatglm3-6b-10k-cosine-model"


/home/cc/.virtualenvs/llm3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = PeftConfig.from_pretrained(peft_model_path_1)

q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float32)

base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')
base_model.requires_grad_(False)
base_model.eval()

/home/cc/.virtualenvs/llm3.9/lib/python3.9/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.91it/s]
/home/cc/.virtualenvs/llm3.9/lib/python3.9/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

## 微调前后效果对比

### ChatGLM-6B

```
输入：

类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领

ChatGLM-6B 微调前输出：

* 版型：修身
* 显瘦：True
* 风格：文艺
* 简约：True
* 图案：印花
* 撞色：True
* 裙下摆：直筒或微喇
* 裙长：中长裙
* 连衣裙：True

ChatGLM-6B 微调后输出：

一款简约而不简单的连衣裙，采用撞色的印花点缀，打造文艺气息，简约的圆领，修饰脸型。衣袖和裙摆的压褶，增添设计感，修身的版型，勾勒出窈窕的身材曲线。
```

### ChatGLM2-6B

```
输入：
类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领

微调前：
这款裙子,版型显瘦,采用简约文艺风格,图案为印花和撞色设计,裙下摆为压褶裙摆,裙长为连衣裙,适合各种场合穿着,让你舒适自在。圆领设计,优雅清新,让你在任何场合都充满自信。如果你正在寻找一款舒适、时尚、优雅的裙子,不妨 考虑这款吧!

微调后: 
这款连衣裙简约的设计，撞色印花点缀，丰富了视觉，上身更显时尚。修身的版型，贴合身形，穿着舒适不束缚。圆领的设计，露出精致锁骨，尽显女性优雅气质。下摆压褶的设计，增添立体感，行走间更显飘逸。前短后长的设计，显 得身材比例更加完美。文艺的碎花设计，更显精致。
```

### ChatGLM3-6B

In [4]:
input_text = '类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领'
print(f'输入：\n{input_text}')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

输入：
类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领


Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [5]:
response, history = base_model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调前：\n{response}')

/home/cc/.virtualenvs/llm3.9/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


ChatGLM3-6B 微调前：
这款连衣裙以其文艺风格和简约设计吸引了我的注意。它采用了印花图案和撞色设计，让整个裙子显得更有活力。压褶的裙下摆和圆领的设计使得这款裙子更加优雅。对于想要显瘦的人来说，这款裙子的版型非常理想，能够充分展现身材的曲线美。总的来说，这是一款非常值得推荐的连衣裙。


In [8]:
liner_model = PeftModel.from_pretrained(base_model, peft_model_path_1)
response, history = liner_model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B liner微调后: \n{response}')

cosine_model = PeftModel.from_pretrained(base_model, peft_model_path_2)
response, history = cosine_model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B cosine微调后: \n{response}')

ChatGLM3-6B liner微调后: 
这一款连衣裙撞色圆领的设计，修饰脖颈精致优雅。简约的版式，穿着大方优雅，特别有文艺范。精美的印花装饰，精致美丽特别出彩。加上压褶的设计，修饰身材显瘦显高。
ChatGLM3-6B cosine微调后: 
连衣裙采用撞色印花图案设计，搭配上精致圆领设计，穿着尽显文艺气息。而其修身版型设计，结合压褶裙摆设计，穿着尽显俏皮可爱气息。简约大方，更显灵动飘逸。


### 评价
从生成内容的相关性、丰富度、语言流畅度和风格适配性来看，两个微调配置的输出各有特点，但 cosine 微调后的结果整体表现更优，具体评价如下：
1. 内容相关性与核心信息覆盖
两者均准确围绕 “连衣裙设计特点” 展开，覆盖了核心元素（撞色、圆领、印花、压褶），未出现偏离主题的内容，相关性均达标。

    linear 微调输出：聚焦 “修饰脖颈、简约版式、文艺范、显瘦显高”，核心卖点明确但维度较集中；
    cosine 微调输出：在覆盖核心元素的基础上，额外区分了 “撞色印花”“修身版型”“压褶裙摆”，对设计细节的拆解更细致（如将 “版型” 和 “裙摆” 分开描述）。

2. 描述丰富度与表达细腻度

    linear 微调输出：语言风格偏平实，用词较基础（如 “精致优雅”“大方优雅”“特别出彩”），重复使用 “优雅” 类词汇，表达稍显单调；对设计效果的描述较直接（“修饰身材显瘦显高”），缺乏层次感。
    cosine 微调输出：用词更丰富多样，新增 “灵动飘逸”“俏皮可爱” 等风格化词汇，通过 “尽显文艺气息”“尽显俏皮可爱气息” 的递进表达，让设计效果的描述更立体；同时通过 “搭配上”“结合” 等连接词，增强了设计元素间的逻辑关联，内容更有层次感。

3. 语言流畅度与风格一致性
两者均无语法错误，句子通顺，但 cosine 微调输出的句式更灵活自然：

    linear 微调输出的句子结构较规整（多为 “XX 设计，修饰 / 穿着 XX” 的短句），节奏偏平缓；
    cosine 微调输出通过长短句结合（如 “连衣裙采用撞色印花图案设计，搭配上精致圆领设计，穿着尽显文艺气息”），语言节奏更生动，且整体风格融合了 “文艺”“俏皮”“灵动” 等多重调性，既符合服饰描述的场景需求，又避免了单调感。

4. 与训练效果的关联性
结合前面对训练配置的分析，cosine 微调（训练 2）在训练损失更低、收敛更充分，这一优势在生成任务中体现为：模型对训练数据中的 “服饰描述逻辑”“风格化词汇搭配” 学习更充分，因此输出内容更细腻、表达更自然。而 linear 微调（训练 1）的输出虽合格，但在细节丰富度和语言灵动性上稍逊，可能与训练中损失下降较平缓、对细节模式的捕捉稍弱有关。


### 结论
cosine 微调后的输出在细节拆解、词汇丰富度、语言流畅度上均优于 linear 微调，更能体现连衣裙设计的层次感和风格特点，与训练阶段 “损失更低、收敛更优” 的结果形成呼应，说明该配置下模型的微调效果更贴合实际生成需求。